# Анализ текстов

### Введение

Метод опорных векторов (Support Vector Machine, SVM) — один из видов линейных классификаторов. Функционал, который он оптимизирует, направлен на максимизацию ширины разделяющей полосы между классами. Из теории статистического обучения известно, что эта ширина тесно связана с обобщающей способностью алгоритма, а ее максимизация позволяет бороться с переобучением.

Одна из причин популярности линейных методов заключается в том, что они хорошо работают на разреженных данных. Так называются выборки с большим количеством признаков, где на каждом объекте большинство признаков равны нулю. Разреженные данные возникают, например, при работе с текстами. Дело в том, что текст удобно кодировать с помощью "мешка слов" — формируется столько признаков, сколько всего уникальных слов встречается в текстах, и значение каждого признака равно числу вхождений в документ соответствующего слова. Ясно, что общее число различных слов в наборе текстов может достигать десятков тысяч, и при этом лишь небольшая их часть будет встречаться в одном конкретном тексте.

Можно кодировать тексты хитрее, и записывать не количество вхождений слова в текст, а TF-IDF. Это показатель, который равен произведению двух чисел: TF (term frequency) и IDF (inverse document frequency). Первая равна отношению числа вхождений слова в документ к общей длине документа. Вторая величина зависит от того, в скольки документах выборки встречается это слово. Чем больше таких документов, тем меньше IDF. Таким образом, TF-IDF будет иметь высокое значение для тех слов, которые много раз встречаются в данном документе, и редко встречаются в остальных.

### Данные

Как мы уже говорили выше, линейные методы часто применяются для решения различных задач анализа текстов. В этом задании мы применим метод опорных векторов для определения того, к какой из тематик относится новость: атеизм или космос.

In [13]:
import numpy as np
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.model_selection import KFold, cross_val_score, GridSearchCV

#### 1 Загрузите объекты из новостного датасета 20 newsgroups, относящиеся к категориям "космос" и "атеизм" 

In [3]:
newgroups = datasets.fetch_20newsgroups(subset='all', categories=['alt.atheism', 'sci.space'])

После выполнения этого кода массив с текстами будет находиться в поле newsgroups.data, номер класса — в поле newsgroups.target.

Одна из сложностей работы с текстовыми данными состоит в том, что для них нужно построить числовое представление. Одним из способов нахождения такого представления является вычисление TF-IDF. В Scikit-Learn это реализовано в классе sklearn.feature_extraction.text.TfidfVectorizer. Преобразование обучающей выборки нужно делать с помощью функции fit_transform, тестовой — с помощью transform.

In [7]:
newgroups.data[0]

'From: 9051467f@levels.unisa.edu.au (The Desert Brat)\nSubject: Re: Keith Schneider - Stealth Poster?\nOrganization: Cured, discharged\nLines: 24\n\nIn article <1pa0f4INNpit@gap.caltech.edu>, keith@cco.caltech.edu (Keith Allan Schneider) writes:\n\n> But really, are you threatened by the motto, or by the people that use it?\n\nEvery time somone writes something and says it is merely describing the norm,\nit is infact re-inforcing that norm upon those programmed not to think for\nthemselves. The motto is dangerous in itself, it tells the world that every\n*true* American is god-fearing, and puts down those who do not fear gods. It\ndoesn\'t need anyone to make it dangerous, it does a good job itself by just\nexisting on your currency.\n\n> keith\n\nThe Desert Brat\n-- \nJohn J McVey, Elc&Eltnc Eng, Whyalla, Uni S Australia,    ________\n9051467f@levels.unisa.edu.au      T.S.A.K.C.            \\/Darwin o\\\nFor replies, mail to whjjm@wh.whyalla.unisa.edu.au      /\\________/\nDisclaimer:

In [8]:
newgroups.target[0]

0

In [10]:
newgroups.target_names

['alt.atheism', 'sci.space']

#### 2 Вычислите TF-IDF-признаки для всех текстов. Обратите внимание, что в этом задании мы предлагаем вам вычислить TF-IDF по всем данным. При таком подходе получается, что признаки на обучающем множестве используют информацию из тестовой выборки — но такая ситуация вполне законна, поскольку мы не используем значения целевой переменной из теста. На практике нередко встречаются ситуации, когда признаки объектов тестовой выборки известны на момент обучения, и поэтому можно ими пользоваться при обучении алгоритма.

In [19]:
vectorizer = TfidfVectorizer()

X = vectorizer.fit_transform(newgroups.data) # count tf-idf
print(X)

y = newgroups.target
feature_map = vectorizer.get_feature_names() # look for words with ith feature

  (0, 12328)	0.0137045393774
  (0, 3083)	0.183545829049
  (0, 16212)	0.130870716213
  (0, 26748)	0.358641401851
  (0, 10446)	0.0877895428146
  (0, 5173)	0.147007441189
  (0, 25602)	0.113127021266
  (0, 9436)	0.160322175265
  (0, 6206)	0.183545829049
  (0, 24745)	0.0137045393774
  (0, 21441)	0.0319711500148
  (0, 15606)	0.179738178431
  (0, 22911)	0.105890061373
  (0, 24461)	0.0896603504628
  (0, 20381)	0.065773758988
  (0, 19110)	0.0143081539732
  (0, 8823)	0.0896603504628
  (0, 9768)	0.0973296270647
  (0, 16346)	0.0137275657721
  (0, 1668)	0.0532164165719
  (0, 14361)	0.0444677816001
  (0, 4890)	0.0212421598026
  (0, 1191)	0.106828889895
  (0, 12512)	0.0593325757435
  (0, 6741)	0.0958731456593
  :	:
  (1785, 8616)	0.0961857077738
  (1785, 11782)	0.055635809034
  (1785, 10058)	0.0742115230561
  (1785, 970)	0.041884777141
  (1785, 16405)	0.0559553567536
  (1785, 28298)	0.0641466708717
  (1785, 8301)	0.0655664206707
  (1785, 13477)	0.0690845681599
  (1785, 11783)	0.0462980883169
  (1785,

#### 3 Подберите минимальный лучший параметр C из множества [10^-5, 10^-4, ... 10^4, 10^5] для SVM с линейным ядром (kernel='linear') при помощи кросс-валидации по 5 блокам. Укажите параметр random_state=241 и для SVM, и для KFold. В качестве меры качества используйте долю верных ответов (accuracy).

Подбор параметров удобно делать с помощью класса sklearn.grid_search.GridSearchCV

Первым аргументом в GridSearchCV передается классификатор, для которого будут подбираться значения параметров, вторым — словарь (dict), задающий сетку параметров для перебора. После того, как перебор окончен, можно проанализировать значения качества для всех значений параметров и выбрать наилучший вариант:

In [21]:
C_grid = {'C': np.power(10.0, np.arange(-5, 6))}
kf = KFold(n_splits=5, shuffle=True, random_state=241)
clf = SVC(kernel='linear', random_state=241)
gs = GridSearchCV(clf, C_grid, scoring='accuracy', cv=kf, n_jobs=-1)
gs.fit(X, y)

GridSearchCV(cv=KFold(n_splits=5, random_state=241, shuffle=True),
       error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=241, shrinking=True,
  tol=0.001, verbose=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'C': array([  1.00000e-05,   1.00000e-04,   1.00000e-03,   1.00000e-02,
         1.00000e-01,   1.00000e+00,   1.00000e+01,   1.00000e+02,
         1.00000e+03,   1.00000e+04,   1.00000e+05])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=0)

#### 4 Обучите SVM по всей выборке с оптимальным параметром C, найденным на предыдущем шаге.

In [26]:
for accuracy in gs.grid_scores_:
    print(accuracy.mean_validation_score, accuracy.parameters) # Оценка качества по кросс-валидации / Значения параметров
    
print('\n-------------------------------\nBest: ', gs.best_estimator_)

0.552631578947 {'C': 1.0000000000000001e-05}
0.552631578947 {'C': 0.0001}
0.552631578947 {'C': 0.001}
0.552631578947 {'C': 0.01}
0.950167973124 {'C': 0.10000000000000001}
0.993281075028 {'C': 1.0}
0.993281075028 {'C': 10.0}
0.993281075028 {'C': 100.0}
0.993281075028 {'C': 1000.0}
0.993281075028 {'C': 10000.0}
0.993281075028 {'C': 100000.0}

-------------------------------
Best:  SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=241, shrinking=True,
  tol=0.001, verbose=False)


/anaconda3/envs/tensorflow/lib/python3.5/site-packages/sklearn/model_selection/_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


In [27]:
clf = gs.best_estimator_
clf.fit(X, y)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=241, shrinking=True,
  tol=0.001, verbose=False)

#### 5 Найдите 10 слов с наибольшим абсолютным значением веса (веса хранятся в поле coef_ у svm.SVC). Они являются ответом на это задание. Укажите эти слова через запятую или пробел, в нижнем регистре, в лексикографическом порядке.

In [43]:
weights = np.absolute(clf.coef_.toarray())

max_weights = sorted(zip(weights[0], feature_map))[-10:]
max_weights.sort(key=lambda x: x[1])
print(max_weights)

[(1.2546899512384038, 'atheism'), (1.2491800073760075, 'atheists'), (1.130612344664901, 'bible'), (1.9203794002294938, 'god'), (1.0970936466401482, 'keith'), (1.2016111817520696, 'moon'), (1.13908083789883, 'religion'), (1.0293069271856938, 'sci'), (1.1801315951388633, 'sky'), (2.6631647884797105, 'space')]


In [50]:
f = open('submission.txt', 'w')
for w, c in max_weights[:-1]:
    f.write(c)
    f.write(',')
f.write(max_weights[-1][1])
f.close()